In [1]:
# Import Python Tools:
from IPython.display import display, Latex
import pandas as pd
import numpy as np
from numpy import sqrt, pi, average, cos, sin, tan, arcsin, arccos, arctan, deg2rad, rad2deg

import load_combos
from steel_design.AISC import aisc
from concrete_design.material_properties import elastic_modulus, cracked_moment_inertia
from concrete_design.steel_reinforcement import rebar_df as rebar
from masonry_design.material_properties import NW_CMU, cmu_elastic_modulus
conc_density = 150.0 #pcf
steel_density = 0.2833 #pci

display(Latex(r"\newpage")) #pagebreak

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


<IPython.core.display.Latex object>

# Concrete Slab Design
## Inputs

In [2]:
# Materials
fc = 3000.0 #psi
fy = 60000.0 #psi

# Dimensions
ln = 24.5 * 12 #in
b = 12.0 #in
t = 9 #in
cover_b = 1 #in
cover_t = 3/4 #in

# Provided Steel
bar = "#6"
Ab = rebar.loc[bar, "Area"]
n = 1 #number of bars per width, b

In [3]:
DL = conc_density * t/12
LL = 40 #psf

LN = ln / 12 #ft
D=DL*b/12
L=LL*b/12
w_u = load_combos.LRFD(D, L) #plf

Mu = w_u * LN**2 / 8 * 12 #lb-in
f"Mu = {round(Mu / 12, 2)} lb-ft"

'Mu = 14931.22 lb-ft'

## Minimum Requirements

In [4]:
# Minimum Thickness if deflection is not calculated
t_min = ln / 30
print(f"t_min (two way) = {round(t_min, 2)} in")

t_min = ln / 20
print(f"t_min (one way) = {round(t_min, 2)} in")

t_min (two way) = 9.8 in
t_min (one way) = 14.7 in


In [5]:
# Minimum Steel
Ag = b * t #in^2
As_min = 0.0018 * Ag
As_minft = As_min * 12 / b

f"As,min = {round(As_min, 3)} in^2 or {round(As_minft, 3)} in^2/ft"

'As,min = 0.194 in^2 or 0.194 in^2/ft'

In [6]:
# Provided Steel
As = n * Ab #in^2
As_ft = As * 12 / b
f"As = {round(As, 3)} in^2 or {round(As_ft, 3)} in^2/ft"

'As = 0.44 in^2 or 0.44 in^2/ft'

In [7]:
# bar spacing
s = b / n
f"bar spacing = {round(s, 2)} in"

'bar spacing = 12.0 in'

## Strength Design

In [8]:
# Compression Steel
bar = "#6"
Ab_c = rebar.loc[bar, "Area"]
n = 1 #number of bars per width, b

As_c = n * Ab_c #compression reinforcement
d_c = t - cover_t

In [9]:
# Required Thickness
phi = 0.9
a = As * fy / (0.85*fc * b)
d = t - cover_b
phiMn = phi * As * fy * (d - a/2) #lb-in

In [10]:
print(f"Mu = {round(Mu / 12, 2)} lb-ft")
print(f"phiMn = {round(phiMn / 12, 2)} lb-ft")
phiMn > Mu

Mu = 14931.22 lb-ft
phiMn = 14985.88 lb-ft


True

## Deflection

### Short-term Deflection

In [11]:
w_a = load_combos.ASD(D, L)

Ma = w_a * LN**2 / 8 * 12 #lb-in
f"Ma = {round(Ma / 12, 2)} lb-ft"

'Ma = 11442.27 lb-ft'

In [12]:
fr = 7.5 * sqrt(fc) #psi
Ig = b * t**3 / 12 #in^4
yt = t / 2 #in
Mcr = fr * Ig / yt #lb-in
f"Mcr = {round(Mcr / 12, 2)} lb-ft"

'Mcr = 5545.69 lb-ft'

In [18]:
# Elastic Modulus
Ec = elastic_modulus(fc=3000) #psi
# Moment of Inertia
Ie = Ig
Icr = cracked_moment_inertia(b=b, h=t, d=d, d_c=d_c, Ec=Ec, As=As, As_c=As_c)
if Ma > 2/3*Mcr:
    Ie = Icr / (1 - ((2/3)*Mcr/Ma)**2 * (1 - Icr/Ig))
else:
    print("section uncracked")

#Elastic Deflection
defl_elastic = 5 * (w_a/12) * ln**4 / (384 * Ec * Ie)
f"Elastic Deflection = {round(defl_elastic, 3)} in"

'Elastic Deflection = 53.319 in'

In [14]:
Icr

2171521574.7353697

### Long-term Deflection

In [15]:
# Provided Steel
bar = "#6"
Ab_c = rebar.loc[bar, "Area"]
n = 1 #number of bars per width, b

# Deflection Factor
xi = 2.0 #duration factor
As_c = n * Ab_c #compression reinforcement
rho_c = As_c / (b * d_c)

lmbda_dlt = xi / (1 + 50*rho_c)
f"lmbda_dlt = {round(lmbda_dlt, 3)}"

'lmbda_dlt = 1.636'

In [16]:
# Total Long-Term Deflection
defl_tot = defl_elastic * lmbda_dlt #in
f"Total Long-Term Deflection = {round(defl_tot, 3)} in"

'Total Long-Term Deflection = 87.25 in'